In [ ]:
import json
import os
from library_dicom.dicom_processor.model.Series import Series 
from library_dicom.dicom_processor.model.SeriesPT import SeriesPT
from library_dicom.dicom_processor.model.SeriesCT import SeriesCT
from library_dicom.dicom_processor.model.csv_reader.MaskBuilder import MaskBuilder

from library_dicom.dicom_processor.tools.folders import *

from library_dicom.dicom_processor.tools.create_mip import *
from library_dicom.dicom_processor.tools.threshold_mask import *

import numpy as np
import SimpleITK as sitk

import csv

In [ ]:
#LIST FROM PARSE_FILTERED_DICOM_DATASET

In [ ]:
#LIST_PATH from json
json_path = '/home/deeplearning/AHL/AHL_new_dataset.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)


In [ ]:
number_of_study = len(dataset)
print("number of study : ", number_of_study)

In [ ]:
#add csv_file to dataset 

csv_path= '/home/deeplearning/AHL/AHL_CSV_DATA/pet0.csv'
csv_file = []
with open(csv_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ';') #liste pour chaque ligne 
    csv_file = []
    for row in reader :
        csv_file.append(row)
        
del csv_file[0] #enlever première ligne

In [ ]:
for data in dataset : 
    uid = data[-3]

    for row in csv_file: 
        if uid in row : 
            data.append(os.path.join( '/home/deeplearning/AHL/AHL_CSV_DATA', row[4]))

In [ ]:
#pet2 & pet 4 

pet_2 = []
pet_4 = []
for data in dataset : 
    if 'pet2' in data[5].lower():
        pet_2.append(data)

for data in dataset : 
    if 'pet4' in data[5].lower():
        pet_4.append(data)


print(len(pet_2))
print(len(pet_4))

In [ ]:
#remove PET 2 and PET 4
remove_study = []
for data in dataset : 
    if len(data) != 8 : 
        remove_study.append(data)

for r in remove_study : 
    dataset.remove(r)

In [ ]:
print('Nombre de study : ', len(dataset))

In [ ]:
#remove remove_bi_file if it exists in every serie 
from library_dicom.dicom_processor.tools.folders import *
directory_validated_dicom = ''
paths = get_series_path(directory_validated_dicom)

for path in paths : 
    remove_bi_file(path)

In [ ]:
#Check if there is Unconstant Spacing error in every serie of the dataset 
unconstant_spacing = []
serie_error = []
for serie in dataset : 
    print(dataset.index(serie))
    try : 
        if serie[1] =='PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            nifti_array = serie_pt_objet.get_numpy_array()
            if serie_pt_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing PET")
                unconstant_spacing.append(serie)
            
            serie_ct_objet = SeriesCT(serie[2])
            nifti_array_ct = serie_ct_objet.get_numpy_array()
            if serie_ct_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing CT")
                unconstant_spacing.append(serie)

        else : 
            serie_pt_objet = SeriesPT(serie[2])
            nifti_array = serie_pt_objet.get_numpy_array()
            if serie_pt_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing PT")
                unconstant_spacing.append(serie)
            serie_ct_objet = SeriesCT(serie[0])
            nifti_array_ct = serie_ct_objet.get_numpy_array()
            if serie_ct_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing CT")
                unconstant_spacing.append(serie)

    except Exception as err : 
        print(serie)
        print(err)
        serie_error.append(serie)


In [ ]:
#Careful : maybe need to rewrite path if the series are in an other folder

In [ ]:
#Generate PET, CT, MASK NIFTI with checking suv_max, mean and sd

nifti_directory = '/home/deeplearning/AHL/AHL_NIFTI'
mip_directory = '/home/deeplearning/AHL/AHL_MIP'

#save serie_path with false mask 
serie_false_mask = []
#save result about serie with false mask 
results_false_mask = []
#save path of MIP to generate PDF 
path_mip = []

#save error serie 
error_dataset = []

for serie in dataset: 
    try : 
        print(dataset.index(serie))
        subliste = []
        if serie[1] == 'PT' : 
            
            serie_pt_objet = SeriesPT(serie[0])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(serie[-1], size_matrix)
            mask_4D = mask_objet.mask_array

            number_roi = mask_4D.shape[3]
            threshold = mask_objet.details_rois['SUVlo']

            if mask_objet.is_correct_suv(nifti_array) == True :  
                print("MASK CORRECT")
                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")

                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[2])
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")

                #generation nifti mask
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                print("EXPORT NIFTI MASK")


            else : 
                results = []
                print("FALSE MASK")
                serie_false_mask.append(serie)

                print(mask_objet.calcul_suv(nifti_array))
                results.append(mask_objet.calcul_suv(nifti_array))

                print(mask_objet.ecart_suv_max(nifti_array))
                results.append(mask_objet.ecart_suv_max(nifti_array))

                print(mask_objet.ecart_suv_mean(nifti_array))
                results.append(mask_objet.ecart_suv_mean(nifti_array))

                print(mask_objet.ecart_SD(nifti_array))
                results.append(mask_objet.ecart_SD(nifti_array))

                results_false_mask.append(results)

                #threshold mask with threshold in csv file  
                mask_4D = threshold_mask(mask_4D, mask_objet.details_rois, nifti_array)
                #create mip for false mask and check 
                angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, 'pet', cmap='Greys', vmin = 0, vmax = 7) 
                subliste.append(angle_filename)
                angle_filename_mask = mip_projection_4D(mask_4D, 0, mip_directory, study_uid, number_roi, cmap='Greys', borne_max = 1.0)
                subliste.append(angle_filename_mask)
                path_mip.append(subliste)


        else : 
            serie_pt_objet = SeriesPT(serie[2])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(serie[-1], size_matrix)
            mask_4D = mask_objet.mask_array
            number_roi = mask_4D.shape[3]
            threshold = mask_objet.details_rois['SUVlo']

            if mask_objet.is_correct_suv(nifti_array) == True : 
                print("MASK CORRECT")
                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")

                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[0])
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")

                #generation nifti mask
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                print("EXPORT NIFTI MASK")

            else : 
                print("FALSE MASK")
                serie_false_mask.append(serie)

                print(mask_objet.calcul_suv(nifti_array))
                results.append(mask_objet.calcul_suv(nifti_array))

                print(mask_objet.ecart_suv_max(nifti_array))
                results.append(mask_objet.ecart_suv_max(nifti_array))

                print(mask_objet.ecart_suv_mean(nifti_array))
                results.append(mask_objet.ecart_suv_mean(nifti_array))

                print(mask_objet.ecart_SD(nifti_array))
                results.append(mask_objet.ecart_SD(nifti_array))

                results_false_mask.append(results)

                #threshold mask with threshold in csv 
                mask_4D = threshold_mask(mask_4D, mask_objet.details_rois, nifti_array)
                #create mip for false mask and check 
                angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, 'pet', cmap='Greys', vmin = 0, vmax = 7) 
                subliste.append(angle_filename)
                angle_filename_mask = mip_projection_4D(mask_4D, 0, mip_directory, study_uid, number_roi, cmap='Greys', borne_max = 1.0)
                subliste.append(angle_filename_mask)
                path_mip.append(subliste)
               

    except Exception as err : 
        print(serie)
        print(err)
        error_dataset.append(serie)

In [ ]:
print("number of error : ", len(error_dataset))
print("number of false mask : " , len(serie_false_mask))

In [ ]:
#Save different json 
directory_nifti = ''
write_json_file(directory_nifti, 'false_mask_serie', serie_false_mask)
write_json_file(directory_nifti, 'false_mask_results', results_false_mask)

write_json_file(directory_nifti, 'error', error_dataset)

mip directory = directory_nifti+'/'+'MIP'
write_json_file(mip_directory, 'path_mip', path_mip)

In [ ]:
#create pdf of MIP for false mask 
filename = os.path.join(mip_directory, 'list_mip_dataset.pdf')
create_pdf_mip(path_mip, filename)

In [ ]:
#Generate PET, CT, MASK NIFTI without checking

nifti_directory = nifti_directory = '/home/deeplearning/AHL/AHL_NIFTI'

error_dataset = []

for serie in dataset: 
    try : 
        print(dataset.index(serie))

        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(serie[-1], size_matrix)
            mask_4D = mask_objet.mask_array

            number_roi = mask_4D.shape[3]

            print("MASK CORRECT")
                    #generation nifti PT
            filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
            print("EXPORT NIFTI PT")

                    #generation nifti CT
            serie_ct_objet = SeriesCT(serie[2])
            serie_ct_objet.get_numpy_array()
            filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
            serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
            print("EXPORT NIFTI CT")

                    #generation nifti mask
            filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
            print("EXPORT NIFTI MASK")
          
          
        else : 
            serie_pt_objet = SeriesPT(serie[2])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(serie[-1], size_matrix)
            mask_4D = mask_objet.mask_array
            number_roi = mask_4D.shape[3]

            print("MASK CORRECT")
                    #generation nifti PT
            filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
            print("EXPORT NIFTI PT")

                     #generation nifti CT
            serie_ct_objet = SeriesCT(serie[0])
            serie_ct_objet.get_numpy_array()
            filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
            serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
            print("EXPORT NIFTI CT")

                    #generation nifti mask
            filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
            print("EXPORT NIFTI MASK")
           
    except Exception as err : 
        print(err)
        print(serie)
        error_dataset.append(serie)

In [ ]:
#Generate PET & CT NIFTI only

nifti_directory = '/home/deeplearning/AHL/AHL_NIFTI/pet4'

error = []
for serie in pet_4 : 
    print(pet_4.index(serie))
    try : 
        if serie[1] == 'PT' : 
                serie_pt_objet = SeriesPT(serie[0]) 
                nifti_array = serie_pt_objet.get_numpy_array()
                study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

                                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
                                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[2])
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")

        else : 
                serie_pt_objet = SeriesPT(serie[2]) 
                nifti_array = serie_pt_objet.get_numpy_array()
                study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

                        
                                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
                                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[0])
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")
 

    except Exception as err : 
        print(serie)
        print(err)
        error.append(serie)

In [ ]:
#Generate PET only

nifti_directory = '/home/deeplearning/AHL/suv_test'
dataset = dataset[0:10]

error = []
for serie in dataset : 
    print(dataset.index(serie))
    print(serie[0])
    try : 
                serie_pt_objet = SeriesPT(serie[1]) 
                nifti_array = serie_pt_objet.get_numpy_array()
                study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

                                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
 

    except Exception as err : 
        print(serie)
        print(err)
        error.append(serie)